*Defines a class for containing the thin wall topographic data and grid data*

In [9]:
import numpy

In [119]:
class Mesh:
    """Describes 2D meshes.
    
    Meshes have shape=(nj,ni) cells with (nj+1,ni+1) vertices with coordinates (x,y).
    
    When constructing, either provide 1d or 2d coordinates (x,y), or assume a
    uniform spherical grid with 'shape' cells covering the whole sphere with
    longitudes starting at x0.

    shape - (nj,ni)
    ni    - number of cells in x-direction (last)
    nj    - number of cells in y-direction (first)
    x     - longitude of mesh (cell corners) (2d)
    y     - latitude of mesh (cell corners) (2d)
    area  - area of cells (2d)
    x0    - used when generating a spherical grid in absence of (x,y)
    """
    def __init__(self, shape=None, x=None, y=None, area=None, x0=-180.):
        if (shape is None) and (x is None) and (y is None): raise Exception('Either shape must be specified or both x and y')
        if (x is None) and (y is not None): raise Exception('Either shape must be specified or both x and y')
        if (x is not None) and (y is None): raise Exception('Either shape must be specified or both x and y')
        # Determine shape
        if shape is not None:
            (nj,ni) = shape
        else: # Determine shape from x and y
            if (x is None) or (y is None): raise Exception('Either shape must be specified or both x and y')
            if len(x.shape)==1: ni = x.shape[0]-1
            elif len(x.shape)==2: ni = x.shape[1]-1
            else: raise Exception('x must be 1D or 2D.')
            if len(y.shape)==1 or len(y.shape)==2: nj = y.shape[0]-1
            else: raise Exception('y must be 1D or 2D.')
        self.ni = ni
        self.nj = nj
        self.shape = (nj,ni)
        # Check shape of arrays and construct 2d coordinates
        if x is not None and y is not None:
            if len(x.shape)==1:
                if len(y.shape)>1: raise Exception('x and y must either be both 1d or both 2d')
                if x.shape[0] != ni+1: raise Exception('x has the wrong length')
            if len(y.shape)==1:
                if len(x.shape)>1: raise Exception('x and y must either be both 1d or both 2d')
                if y.shape[0] != nj+1: raise Exception('y has the wrong length')
            if len(x.shape)==2 and len(y.shape)==2:
                if x.shape != y.shape: raise Exception('x and y are 2d and must be the same size')
                if x.shape != (nj+1,ni+1): raise Exception('x has the wrong size')
                self.x = x
                self.y = y
            else:
                self.x, self.y = numpy.meshgrid(x,y)
        else: # Construct coordinates
            y1d = numpy.linspace(-90.,90.,nj+1)
            x1d = numpy.linspace(x0,x0+360.,ni+1)
            self.x, self.y = numpy.meshgrid(x1d,y1d)
        if area is not None:
            if area.shape != (nj,ni): raise Exception('area has the wrong shape or size')
            self.area = area
    def rhumb_length(p1,p2):
        """Distance between two points, p1 and p2, following a constant bearing.
        Positions are given as (latitude,longitude) tuples measured in degrees."""
        deg2rad = numpy.pi/180.
        phi1 = deg2rad * p1[0]
        phi2 = deg2rad * p2[0]
        dphi = phi2 - phi1
        dlambda2 = ( deg2rad * ( p2[1] - p1[1] ) )**2
        dpsi = numpy.log( numpy.tan( 0.25 * numpy.pi + 0.5*phi2 ) / numpy.tan( 0.25*numpy.pi + 0.5*phi1 ) )
        if numpy.abs(dphi)==0: q2 = numpy.cos( 0.5*( phi1 + phi2 ) )**2
        else: q2 = ( dphi / dpsi )**2
        return numpy.sqrt( dphi**2 + q2 * dlambda2 )
    def angle_p1p2(p1,p2):
        """Angle at center of sphere between two points on the surface of the sphere.
        Positions are given as (latitude,longitude) tuples measured in degrees."""
        deg2rad = numpy.pi/180.
        phi1 = deg2rad * p1[0]
        phi2 = deg2rad * p2[0]
        dphi_2 = 0.5 * ( phi2 - phi1 )
        dlambda_2 = 0.5 * deg2rad * ( p2[1] - p1[1] )
        a = numpy.sin( dphi_2 )**2 + numpy.cos( phi1 ) * numpy.cos( phi2 ) * ( numpy.sin( dlambda_2 )**2 )
        c = 2. * numpy.arctan2( numpy.sqrt(a), numpy.sqrt( 1. - a ) )
        return c
    def great_arc_length(p1,p2,R=1.):
        """Shortest distance between two points, p1 and p2.
        Positions are given as (latitude,longitude) tuples measured in degrees."""
        return R*Mesh.angle_p1p2(p1,p2)
    def angle_triangle(pA,pB,pC):
        """Angle ABC (angle between AC at B)
        Positions are given as (latitude,longitude) tuples measured in degrees."""
        deg2rad = numpy.pi/180.
        a = Mesh.angle_p1p2(pA,pB)
        b = Mesh.angle_p1p2(pB,pC)
        c = Mesh.angle_p1p2(pC,pA)
        cos_c = numpy.cos(c)
        cos_b = numpy.cos(b)
        sin_b = numpy.sin(b)
        cos_a = numpy.cos(a)
        sin_a = numpy.sin(a)
        return numpy.arccos( ( cos_c - cos_b*cos_a) / (sin_a * sin_b) )
    def area_spherical_triangle(p1,p2,p3):
        """Area of a spherical triangle defined by great arcs between three positions on
        the surface of a sphere.
        Positions are given as (latitude,longitude) tuples measured in degrees."""
        A = Mesh.angle_triangle(p1,p2,p3)
        B = Mesh.angle_triangle(p2,p3,p1)
        C = Mesh.angle_triangle(p3,p1,p2)
        return (A+B+C-numpy.pi)
    def area_ggr_triangle(pA,pB,pC, n=5):
        """Area of a triangle ABC where AB and BC are great arcs and AC is a rhumb
        line (or straight line in geographic coordinates).
        Positions are given as (latitude,longitude) tuples measured in degrees."""
        area = 0.*pB[0]
        for j in range(n):
            sL = float(j)/float(n)
            pL = ((1.-sL)*pA[0] + sL*pC[0], (1.-sL)*pA[1] + sL*pC[1])
            sR = float(j+1)/float(n)
            pR = ((1.-sR)*pA[0] + sR*pC[0], (1.-sR)*pA[1] + sR*pC[1])
            area += Mesh.area_spherical_triangle(pR,pB,pL)
        return area
    def __area_grid_cell__(p1,p2,p3,p4):
        pass

M = Mesh(shape=(5,3))
print( Mesh.rhumb_length((0,0),(90,0))/numpy.pi, 0.5)
print( Mesh.rhumb_length((0,0),(0,90))/numpy.pi, 0.5)
print( Mesh.great_arc_length((0,0),(90,0))/numpy.pi, 0.5)
print( Mesh.great_arc_length((0,0),(0,90))/numpy.pi, 0.5)
print( Mesh.angle_p1p2((0,0),(90,0))/numpy.pi*180., 90.)
print( Mesh.angle_triangle( (1,0), (0,0), (0,90) )/numpy.pi*180., 90.)
print( Mesh.area_spherical_triangle( (0,0), (90,0), (0,90) )/numpy.pi*8, 4.)
print( Mesh.area_ggr_triangle( (0,0), (90,0), (0,90), n=2 )/numpy.pi*8, 4. )
print( Mesh.area_ggr_triangle( (1,0), (0,0), (0,1), n=1 )/numpy.pi*8, 4., 1)
print( Mesh.area_ggr_triangle( (1,0), (0,0), (0,1), n=2 )/numpy.pi*8, 4., 2)
print( Mesh.area_ggr_triangle( (1,0), (0,0), (0,1), n=5 )/numpy.pi*8, 4., 5)
print( Mesh.area_ggr_triangle( (1,0), (0,0), (0,1), n=20 )/numpy.pi*8, 4., 20)
print( Mesh.area_ggr_triangle( (1,0), (0,0), (0,1), n=200 )/numpy.pi*8, 4., 200)
print( Mesh.area_ggr_triangle( (1,0), (0,0), (0,1), n=2000 )/numpy.pi*8, 4., '2k')
print( Mesh.area_ggr_triangle( (1,0), (0,0), (0,1), n=5000 )/numpy.pi*8, 4., '5k')
#print( Mesh.area_spherical_triangle( (45,0), (0,0), (45,45) )/numpy.pi*8 )
#print( Mesh.area_rhumb_triangle( (45,0), (0,0), (45,45) )/numpy.pi*8 )

0.5 0.5
0.5 0.5
0.5 0.5
0.5 0.5
90.0 90.0
90.0 90.0
4.0 4.0
4.0 4.0
0.000387870637108 4.0 1
0.000387848484977 4.0 2
0.000387842283732 4.0 5
0.000387841220132 4.0 20
0.000387843909749 4.0 200
0.000387796099978 4.0 2k
0.000387084130943 4.0 5k


In [ ]:
class ThinWall(Mesh):
    """Container for thin wall topographic data and mesh/grid data

    zc_mean - mean elevation of cell (2d)
    zu_mean - mean elevation of western edge of cell (2d)
    zv_mean - mean elevation of southern edge of cell (2d)
    """
    def __init__(self, shape):
        self.shape = shape
        self.ni = shape[1]
        self.nj = shape[0]
        self.x = x
        self.y = y
        self.area = area
        

In [130]:
Mesh.angle_p1p2((0,0),(10,10))/numpy.pi*90*numpy.sqrt(2)

9.9744795523640573